<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/Sharpe_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

샤프지수 = (포트폴리오 수익률 - 무위험금리)/포트폴리오 변동성<BR>
샤프지수를 함수로 작성하여, 주식과 주가지수 그룹에 대해 구해본다.<BR>
우선 Finance Data Reader를 설치한다.

In [ ]:
pip install -q finance-datareader

필요한 모듈을 임포트 한다.

In [3]:
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

관심있는 주식의 목록과 회사명을 작성한다.

In [4]:
stocks = ['005930', '066570', '035420', '051910', '017670' ] #symbol
names = ['Samsung', 'LGE',    'Naver', 'LG-Chem', 'SK-Tel'] # Name

샤프를 검색할 기간을 설정한다.

In [5]:
start = "2015-01-01"
end = "2022-12-31"

연납화를 하기 위한 factor를 설정한다. 미국은 통상 252로 설정한다.

In [6]:
SCALE_FACTOR = 252

무위험 금리를 설정한다.

In [7]:
RISKFREE = 0.01

샤프지수를 구하는 함수를 정의한다.<BR>
전체기간을 연간으로 나누어 연간 샤프지수를 구하는 함수와 전체기간을 고려한 샤프지수를 구하는 2가지 기능을 가지며, 각각 'Y'와 'A'로 구분한다.



In [23]:
def GetSharpe( d_ret , columns, freq='Y', riskfree=0):
  if freq == 'Y' :  # Annual Sharpe
    a_sharpe = pd.DataFrame()
    a_ret = pd.DataFrame()
    a_std = pd.DataFrame()

    for stock in columns:
      a_ret[stock] = d_ret[stock].resample('YE').mean() * SCALE_FACTOR
      a_std[stock] = d_ret[stock].resample('YE').std()*np.sqrt(SCALE_FACTOR)
      a_sharpe = (a_ret - riskfree) / a_std

    a_sharpe.columns = columns
    return a_sharpe

  elif freq == 'H':   # Historical Sharpe
    # Sharpe from historical data
    avg_ret = np.zeros( len(columns))
    avg_std = np.zeros( len(columns))
    sharpe = np.zeros( len(columns))

    for i, stock in enumerate(columns):
      avg_ret[i] = d_ret[stock].mean() * SCALE_FACTOR
      avg_std[i] = d_ret[stock].std() * np.sqrt(SCALE_FACTOR)
      sharpe[i] = (avg_ret[i] - riskfree)/ avg_std[i]

    return sharpe, avg_ret, avg_std

주가와 일별 수익률을 저장할 데이터프레임을 설정한다.

In [24]:
df = pd.DataFrame()
daily_r = pd.DataFrame()

주가를 읽어오면서, 일별 수익률을 계산하여 저장한다.

In [25]:
for stock in stocks:
  df[stock] = fdr.DataReader( stock, start, end ).Close
  daily_r[stock] = df[stock].pct_change()

가독성을 위해, 티커 번호 대신 회사명으로 열을 대체한다.

In [27]:
df.columns = names
daily_r.columns = names

주가의 흐름을 그래프로 살펴본다.

In [ ]:
df.plot()

수익률의 변화도 살펴본다.

In [ ]:
daily_r.plot()

먼저 연간 샤프지수를 구해본다.

In [30]:
y_sharpe = GetSharpe( daily_r, names, 'Y', RISKFREE)

샤프지수를 출력해 본다.

In [ ]:
print( y_sharpe )

통계량을 출력해 본다.

In [ ]:
y_sharpe.describe()

연간변화를 그래프로 그려본다.

In [ ]:
y_sharpe.plot()

과거 18년간의 누적  데이터에 근거한 연간 샤프지수를 구해 본다.

In [34]:
h_sharpe, h_ret, h_std = GetSharpe( daily_r, names, 'H', RISKFREE)

18년간 누적 데이터에 기반한 연평균 표준편차와 수익률 그리고 샤프지수를 3차원 산점도로 그려본다.<BR>X축: 샤프, Y축: 수익률, Z축: 변동성

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 주어진 데이터

# Matplotlib에서 3D 그래프를 그리기 위한 플롯 생성
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')

# 데이터를 3D 공간에 플롯하고 레이블 표시
for i in range(len(names)):
    ax.scatter(h_sharpe[i], h_ret[i], h_std[i], marker='o')
    ax.text(h_sharpe[i], h_ret[i], h_std[i], f"{names[i]}\n({h_sharpe[i]:.2f},{h_ret[i]:.2f},{h_std[i]:.2f})", fontsize=12, ha='center')

# 축 레이블 설정
ax.set_xlabel('Sharpe')
ax.set_ylabel('Return')
ax.set_zlabel('Volatility')


# 그래프 표시
plt.show()


이번에는 yfinance를 이용해 5가지 주가지수를 가지고 샤프지수를 구해 본다.

In [ ]:
import yfinance as yf

idx_tickers = [ "^KS11", "^GSPC" , "^DJI" , "^NDX", "^HSI"]
idx_names = [ "KOSPI", "S&P500", "Dow-Jones" , "Rusell2000", "Hang Seng"]

start_date = "2018-01-01"
end_date = "2022-12-31"

idx_df = yf.download( idx_tickers , start_date, end_date, progress=False).Close
idx_ret = idx_df.pct_change()

idx_df.columns = idx_names
idx_ret.columns = idx_names

idx_y_sharpe = GetSharpe( idx_ret, idx_names, 'Y', RISKFREE )

샤프지수를 출력해 본다.

In [44]:
print( idx_y_sharpe )

               KOSPI    S&P500  Dow-Jones  Rusell2000  Hang Seng
Date                                                            
2018-12-31 -0.312361 -0.398116  -0.802899   -1.363253  -0.056692
2019-12-31  1.579377  1.987205   0.558106    0.567382   1.962103
2020-12-31  0.341794  0.571383  -0.077053    1.043413   1.204691
2021-12-31  1.339272  1.777227  -0.703421    0.232739   1.289598
2022-12-31 -0.411574 -0.805652  -0.384693   -1.493343  -1.085706


통계량을 출력해 본다.

In [ ]:
idx_y_sharpe.describe()

샤프지수를 도식화 해본다.

In [ ]:
idx_y_sharpe.plot()

5년치 데이터의 누적 수익률, 표준편차, 샤프를 구해 본다.



In [40]:
idx_h_sharpe, idx_h_ret, idx_h_std = GetSharpe( idx_ret, idx_names, 'H', RISKFREE)

adjustText를 설치한다. <BR>matplotlib 그래프에서 텍스트 라벨이 겹치지 않도록 자동으로 조정해주는 유용한 도구

In [47]:
pip install --q adjustText

In [ ]:
# 주어진 데이터
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from adjustText import adjust_text

# Matplotlib에서 3D 그래프를 그리기 위한 플롯 생성
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')  # (111) = (행,열. 인덱스)

for i in range(len(idx_names)):
    ax.scatter( idx_h_sharpe[i], idx_h_ret[i],idx_h_std[i], marker='o')
    ax.text( idx_h_sharpe[i], idx_h_ret[i], idx_h_std[i],   f"{idx_names[i]}\n({idx_h_sharpe[i]:.2f},{idx_h_ret[i]:.2f},{idx_h_std[i]:.2f})", fontsize=12, ha='center')

# 축 레이블 설정
ax.set_xlabel('X: Sharpe')
ax.set_ylabel('Y: Return')
ax.set_zlabel('Z: Volatility')

# 그래프 표시
plt.show()